# Cloudformation Stack Deletion

* This step is responsible to delete the cloudformation stack that was created as the first step of the process.

### Importing Required Libraries ::
* __boto3__: Required to connect as operate AWS task
* __botocore__: Required to handle the exceptions related to boto3 tasks
* __json__: To convert python native dictionaries to string, to write in files
* __pickle__: To store configuration dictionary which will be used in next notebooks
* __os__: To do os related jobs like checking for file existance

In [1]:
import boto3, botocore
import json, pickle, os
from botocore.exceptions import ClientError

### Reading user defined configurations & Declaring other variables::
* User is allowed to provide specific configurations using provided format of configuration file. If user does not provide any confguration or provides wrong configuration format, then default values will be used. Please check **README.MD** file for default values.

In [2]:
try:
    with open("cluster_config.json", "r") as config_file:
        user_config = json.load(config_file)
    user = user_config.get('UserName', "root")
    region = user_config.get('Region', "us-east-1")
    wrk_spc_dir = user_config['WorkspaceDirectory']
    pickle_file = wrk_spc_dir + "/SparkClusterOnAWSEC2_" + user + "_CurrentStatus.pkl"
    if os.path.exists(pickle_file):
        with open(pickle_file, 'rb') as pickle_handle:
            user_config = pickle.load(pickle_handle)
        stack_id = user_config['StackId']
    else:
        print("Status file '"+ pickle_file + "' is not available, which is unexpected. Please start running from 'cloudformation_stack_creation.ipynb' file.")
        raise Exception
except Exception as e:
    print("Unexpected error while deleting Cloudformation Stack: " + str(e))
    exit()

### Creating boto3 session, clients and resources::
These resources will be used to connect and run different tasks on AWS.

In [3]:
try:
    session = boto3.session.Session(region_name=region)
    cf_client = session.client('cloudformation')
except Exception as e:
    print("Unexpected error while creating boto3 session, client and resources: " + str(e))
    exit()

### Issuing deletion command to the Stack::
* Deletion command is executed on the stack using delete_stack API of boto3 library.

In [4]:
try:
    cf_client.delete_stack(StackName=stack_id)
    print("Deletion process initiated. It will take 3-4 minutes based on the network and stack size.")
except Exception as e:
    print("Unexpected error while deleting Cloudformation Stack: " + str(e))
    exit()

Deletion process initiated. It will take 3-4 minutes based on the network and stack size.
